In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 1. 데이터 불러오기
df = pd.read_csv('../../data/secom.csv')

# Time 파싱
df['Time'] = pd.to_datetime(df['Time'])
df['Hour'] = df['Time'].dt.hour
df['DayOfWeek'] = df['Time'].dt.dayofweek
df['YearMonth'] = df['Time'].dt.to_period('M')

target_col = 'Pass/Fail'

print(f"   샘플 수: {len(df):,}개")
print(f"   기간: {df['Time'].min()} ~ {df['Time'].max()}")

   샘플 수: 1,567개
   기간: 2008-01-08 02:02:00 ~ 2008-12-10 18:47:00


In [4]:
# 2. 양품률 계산

# 전체 양품률
total_defect_rate = (df[target_col] == 1).sum() / len(df)
total_quality_rate = 1 - total_defect_rate

print(f"\n전체 양품률: {total_quality_rate:.2%}")
print(f"  정상: {(df[target_col] == -1).sum():,}개 ({(1-total_defect_rate):.1%})")
print(f"  불량: {(df[target_col] == 1).sum():,}개 ({total_defect_rate:.1%})")

# 시간대별 양품률
print("\n시간대별 양품률:")
print("-" * 40)

hourly_quality = df.groupby('Hour')[target_col].apply(
    lambda x: (x == -1).sum() / len(x)  # 정상(-1)의 비율
)

for hour, quality in hourly_quality.items():
    print(f"{hour:2d}시: {quality:.2%}")

# 최고/최저
best_hour_q = hourly_quality.idxmax()
worst_hour_q = hourly_quality.idxmin()
print(f"\n최고: {best_hour_q}시 ({hourly_quality.max():.2%})")
print(f"최저: {worst_hour_q}시 ({hourly_quality.min():.2%})")
print(f"차이: {(hourly_quality.max() - hourly_quality.min()):.2%}p")


전체 양품률: 93.36%
  정상: 1,463개 (93.4%)
  불량: 104개 (6.6%)

시간대별 양품률:
----------------------------------------
 0시: 95.38%
 1시: 92.86%
 2시: 95.00%
 3시: 92.86%
 4시: 90.77%
 5시: 94.59%
 6시: 91.94%
 7시: 95.89%
 8시: 95.31%
 9시: 89.09%
10시: 98.11%
11시: 92.65%
12시: 95.52%
13시: 90.00%
14시: 96.10%
15시: 88.89%
16시: 93.10%
17시: 96.55%
18시: 95.24%
19시: 93.65%
20시: 92.41%
21시: 90.14%
22시: 92.86%
23시: 93.33%

최고: 10시 (98.11%)
최저: 15시 (88.89%)
차이: 9.22%p


In [ ]:
# 3. 가동률 계산
# 가동률 = (전체 시간 - 가동 중단 시간) / 전체 시간
# 가동 중단: 데이터가 기록되지 않은 시간

# 시간대별 샘플 수
hourly_counts = df.groupby('Hour').size()
print(f"시간대별 데이터:")
for hour, count in hourly_counts.items():
    pct = count / len(df) * 100
    print(f"{hour:2d}시: {count:3d}개 ({pct:4.1f}%)")

data_coverage = len(df['Time'].unique()) / ((df['Time'].max() - df['Time'].min()).days * 24)
print(f"  → 데이터 커버리지: {data_coverage:.1%}")

# 결론: 가동률 85% 가정 (실제 정지시간 데이터 없어 업계 평균 사용)
# -> 더 정확한 계산 위해서는 설비 로그 데이터 필요 (현재 데이터는 샘플링데이터 이므로 신뢰도 낮음)

시간대별 데이터:
 0시:  65개 ( 4.1%)
 1시:  42개 ( 2.7%)
 2시:  60개 ( 3.8%)
 3시:  70개 ( 4.5%)
 4시:  65개 ( 4.1%)
 5시:  74개 ( 4.7%)
 6시:  62개 ( 4.0%)
 7시:  73개 ( 4.7%)
 8시:  64개 ( 4.1%)
 9시:  55개 ( 3.5%)
10시:  53개 ( 3.4%)
11시:  68개 ( 4.3%)
12시:  67개 ( 4.3%)
13시:  50개 ( 3.2%)
14시:  77개 ( 4.9%)
15시:  99개 ( 6.3%)
16시:  58개 ( 3.7%)
17시:  58개 ( 3.7%)
18시:  63개 ( 4.0%)
19시:  63개 ( 4.0%)
20시:  79개 ( 5.0%)
21시:  71개 ( 4.5%)
22시:  56개 ( 3.6%)
23시:  75개 ( 4.8%)
  → 데이터 커버리지: 19.0%
